# Scratch

In [1]:
EMPFILE = "/Users/adarshrp/Projects/kona/data/emp.csv"
emp = spark.read.option("header", True).option("inferSchema", True).csv(EMPFILE)
emp.printSchema()
emp.createOrReplaceTempView("emp")


root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- emp_dept_id: integer (nullable = true)



In [3]:
spark.sql("""
select sum(age + 10) / count(*) from EMP
group by emp_dept_id
""").show(10)

+----------------------------------------------------------------------------+
|(CAST(sum(CAST((age + 10) AS BIGINT)) AS DOUBLE) / CAST(count(1) AS DOUBLE))|
+----------------------------------------------------------------------------+
|                                                                        47.0|
|                                                          42.666666666666664|
|                                                                       41.75|
|                                                                        47.4|
|                                                          54.333333333333336|
+----------------------------------------------------------------------------+



In [ ]:


spark.sql("""
WITH CTE1 AS (select '--cte1', age, '--cte1' from emp),
 CTE2 AS (select '--cte2', age, '--cte2' from emp)
SELECT * FROM CTE1, (select '2nd', dept_id, name from EMP) CTE1
WHERE name in (select name from emp CTE1 where CTE1.age >= 0) 
and age in (select CTE2.age from EMP CTE2 where CTE1.age = CTE2.age)
""").show(100)

In [35]:
parquetFile = spark.read.parquet("/tmp/file.parquet")
parquetFile.createOrReplaceTempView("parquetFile")

In [36]:
parquetFile

DataFrame[column_1: bigint, column_2: string, column_3: string]

In [37]:
parquetFile.columns

['column_1', 'column_2', 'column_3']

In [38]:
spark.sql("SELECT * FROM parquetFile").show(100)

+--------+-----------+--------------------+
|column_1|   column_2|            column_3|
+--------+-----------+--------------------+
|       1|    AMERICA|hs use ironic, ev...|
|       2|       ASIA|ges. thinly even ...|
|       3|     EUROPE|ly final courts c...|
|       4|MIDDLE EAST|uickly special ac...|
+--------+-----------+--------------------+



In [41]:
parquetFile.rdd.getNumPartitions()

1

In [42]:
parquetFile.write.format("parquet").partitionBy("column_2").mode("overwrite").save("/tmp/pqptns.parquet")

### Read TPCH Data

In [8]:
!ls ../data

Convert.ipynb     datatypes.csv     emp.parquet       userdata1.parquet
R.csv             dept.csv          empdir
S.csv             dept_details.csv  northwind
T.csv             emp.csv           tpch0.01


In [16]:
parquetFile = spark.read.parquet("/Users/adarshrp/Downloads/part-0.parquet")
parquetFile.createOrReplaceTempView("LINEITEM")

In [17]:
parquetFile

DataFrame[l_orderkey: bigint, l_partkey: bigint, l_suppkey: bigint, l_linenumber: int, l_quantity: double, l_extendedprice: double, l_discount: double, l_tax: double, l_returnflag: string, l_linestatus: string, l_shipdate: date, l_commitdate: date, l_receiptdate: date, l_shipinstruct: string, l_shipmode: string, l_comment: string]

In [18]:
sql = """
SELECT L_RETURNFLAG, L_LINESTATUS, SUM(L_QUANTITY) AS SUM_QTY, MAX(L_LINESTATUS), MIN(L_LINESTATUS)
FROM LINEITEM
--WHERE L_SHIPDATE <= CAST('1992-01-03' AS DATE32)
GROUP BY L_RETURNFLAG, L_LINESTATUS;
"""

In [19]:
spark.sql(sql).show(100)

23/08/06 19:27:33 ERROR Executor: Exception in task 2.0 in stage 14.0 (TID 207)
java.lang.RuntimeException: native zStandard library not available: this version of libhadoop was built without zstd support.
	at org.apache.hadoop.io.compress.ZStandardCodec.checkNativeCodeLoaded(ZStandardCodec.java:65)
	at org.apache.hadoop.io.compress.ZStandardCodec.getDecompressorType(ZStandardCodec.java:211)
	at org.apache.hadoop.io.compress.CodecPool.getDecompressor(CodecPool.java:181)
	at org.apache.parquet.hadoop.CodecFactory$HeapBytesDecompressor.<init>(CodecFactory.java:98)
	at org.apache.parquet.hadoop.CodecFactory.createDecompressor(CodecFactory.java:210)
	at org.apache.parquet.hadoop.CodecFactory.getDecompressor(CodecFactory.java:199)
	at org.apache.parquet.hadoop.CodecFactory.getDecompressor(CodecFactory.java:42)
	at org.apache.parquet.hadoop.ParquetFileReader$Chunk.readAllPages(ParquetFileReader.java:1020)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextRowGroup(ParquetFileReader.java

Py4JJavaError: An error occurred while calling o62.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 14.0 failed 1 times, most recent failure: Lost task 3.0 in stage 14.0 (TID 208) (localhost executor driver): java.lang.RuntimeException: native zStandard library not available: this version of libhadoop was built without zstd support.
	at org.apache.hadoop.io.compress.ZStandardCodec.checkNativeCodeLoaded(ZStandardCodec.java:65)
	at org.apache.hadoop.io.compress.ZStandardCodec.getDecompressorType(ZStandardCodec.java:211)
	at org.apache.hadoop.io.compress.CodecPool.getDecompressor(CodecPool.java:181)
	at org.apache.parquet.hadoop.CodecFactory$HeapBytesDecompressor.<init>(CodecFactory.java:98)
	at org.apache.parquet.hadoop.CodecFactory.createDecompressor(CodecFactory.java:210)
	at org.apache.parquet.hadoop.CodecFactory.getDecompressor(CodecFactory.java:199)
	at org.apache.parquet.hadoop.CodecFactory.getDecompressor(CodecFactory.java:42)
	at or